<a href="https://colab.research.google.com/github/dsevero/generative-models/blob/master/experiments/GAN/notebooks/goodfellow_2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Nets (Goodfellow 2014)
https://arxiv.org/pdf/1406.2661.pdf

In [0]:
# necessary for autograd compatibility
!pip install scipy==1.1.0 -q

# update tensorflow
!pip install tensorflow==2.0.0b1 -q

In [0]:
import autograd.numpy as np
import autograd.numpy.random as npr
import warnings
import tensorflow as tf
from autograd import grad
from autograd.misc import flatten
    

warnings.simplefilter(action='ignore', category=FutureWarning)
π = np.pi

## Definitions and Tests

In [0]:
# reference: 
# https://github.com/HIPS/autograd/blob/master/examples/generative_adversarial_net.py

def relu(x):       
    return np.maximum(0, x)

def sigmoid(x):    
    return 0.5 * (np.tanh(x) + 1.0)

def logsigmoid(x): 
    return x - np.logaddexp(0, x)

def init_random_params(scale, layer_sizes, rs=npr.RandomState(0)):
    """Build a list of (weights, biases) tuples,
       one for each layer in the net."""
    return [(scale * rs.randn(m, n),   # weight matrix
             scale * rs.randn(n))      # bias vector
            for m, n in zip(layer_sizes[:-1], layer_sizes[1:])]

def batch_normalize(activations):
    mbmean = np.mean(activations, axis=0, keepdims=True)
    return (activations - mbmean) / (np.std(activations, axis=0, keepdims=True) + 1)

def neural_net_predict(params, inputs, use_batch_norm=True):
    """Params is a list of (weights, bias) tuples.
       inputs is an (N x D) matrix."""
    inpW, inpb = params[0]
    inputs = relu(np.dot(inputs, inpW) + inpb)
    for W, b in params[1:-1]:
        outputs = np.dot(inputs, W) + b
        if use_batch_norm:
            outputs = batch_normalize(outputs)
        inputs = relu(outputs)
    outW, outb = params[-1]
    outputs = np.dot(inputs, outW) + outb
    return outputs

def generate_from_noise(gen_params, num_samples, noise_dim, rs):
    noise = rs.rand(num_samples, noise_dim)
    samples = neural_net_predict(gen_params, noise)
    return sigmoid(samples)

In [0]:
params = 2*((np.eye(3), np.zeros(3)),)

assert (batch_normalize([0, 0, 0]) == [0, 0, 0]).all()
assert (batch_normalize([-1, 1]) == [-0.5, 0.5]).all()
assert (batch_normalize([[-1, 1], [-2, 2]]) == [[1/3, -1/3], [-1/3, 1/3]]).all()

In [57]:
x = np.ones((3,3)).dot()
neural_net_predict(params, x, False)

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [62]:
np.ones((3,3))

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])